In [2]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5): 
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [3]:
# import keras
# import numpy as np
# path = '/home/jovyan/dsc650/data/external/enron/davis-d/sent_items/1_'
# text = open(path).read().lower()
# print('Corpus length:', len(text))

In [4]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 0s 1us/step
Corpus length: 600893


In [5]:
maxlen = 60                                                            
step = 3                                                               

sentences = []                                                         

next_chars = []                                                        

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))                                        
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)       

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)      
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)              
for i, sentence in enumerate(sentences):                               
    for t, char in enumerate(sentence):                                
        x[i, t, char_indices[char]] = 1                                
    y[i, char_indices[next_chars[i]]] = 1            

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [6]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [7]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
import random
import sys

for epoch in range(1,20):                                          
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)                       
    start_index = random.randint(0, len(text) - maxlen - 1)         
    generated_text = text[start_index: start_index + maxlen]        
    print('--- Generating with seed: "' + generated_text + '"')     
    for temperature in [0.2, 0.5, 1.0, 1.2]:                        
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):                                        
            sampled = np.zeros((1, maxlen, len(chars)))             
            for t, char in enumerate(generated_text):               
                sampled[0, t, char_indices[char]] = 1.              

            preds = model.predict(sampled, verbose=0)[0]            
            next_index = sample(preds, temperature)                 
            next_char = chars[next_index]                           

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

epoch 1
1565/1565 [==============================] - 199s 127ms/step - loss: 1.8230
--- Generating with seed: "s state of self contempt, of conscience pangs, of
despair in"
------ temperature: 0.2
s state of self contempt, of conscience pangs, of
despair in the superviled the despress of the superfice and in the supernication of the such and the world the are strenger the such the such be the such and the such and the despress of the such the superfice of the condection of the superfice and of the such a sunders and the superfice of the such the say the supervile of the superfice of the supervility of the such the spirit, and the such and the such a------ temperature: 0.5
ervility of the such the spirit, and the such and the such a strange the precisely the still the grounder the means and for so the soul the "soul precenting have one neverstand into be the world and which a well to all the barity of the devilonate be astimation, the putes and
species is always one supertion of presist

<ipython-input-8-66c6eecb696d>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


 very inner and a doubt to conwarsifiesm, prejudicles--the
sign of the pustese
for upon hereganciation,
and he gratus and
besiennes,
more exactide for on the
spirits of when the conpentire of smir byy
"what manifounted too has old evengked as now having "nutilimebl" and of a trentation of a squenday humment and must are for the spends of fundamenta------ temperature: 1.2
a squenday humment and must are for the spends of fundamentally reallessely dewactly highly above so effice everything timent conceptting the sphened the findinging the finer our imolutive influence and what it has only known thereforee toistic, give peoplee--"ore is virbue for the
spirit? the, wame a tasib altfur,. with a religior.
for,'men negy foo
europear
spur as injure, of a mi sjout highesty before in the eiris, to man; he isway: therefore
simplifif,epoch 18
1565/1565 [==============================] - 188s 120ms/step - loss: 1.3187
--- Generating with seed: " as the
house of a god, rears itself vastly into the v